In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd

import pickle

import os

from tqdm.notebook import tqdm

import uproot

from collections import defaultdict


In [2]:
# this code contains some annoying/slow things to do matching myself
# potentially using medulla is a better option


In [3]:
def get_df_from_file(file_name):

    f = uproot.open('/nevis/riverside/data/leehagaman/spine/flat_cafs/' + file_name)

    dic = {}
    dic.update(f["recTree"].arrays(["rec.dlp_true.nu_id", "rec.dlp_true.is_fiducial", "rec.dlp_true.particles.interaction_id", "rec.dlp_true.particles.match_ids", "rec.dlp.particles.id", 
                                    "rec.dlp_true.particles.pdg_code", "rec.dlp.particles.pdg_code",
                                    "rec.dlp_true.particles.is_contained", "rec.dlp.particles.is_contained",
                                    "rec.dlp_true.particles.is_primary", "rec.dlp.particles.is_primary",
                                    "rec.dlp_true.particles.start_point.0", "rec.dlp_true.particles.start_point.1", "rec.dlp_true.particles.start_point.2", 
                                    "rec.dlp.particles.start_point.0", "rec.dlp.particles.start_point.1", "rec.dlp.particles.start_point.2",
                                    "rec.dlp_true.particles.end_point.0", "rec.dlp_true.particles.end_point.1", "rec.dlp_true.particles.end_point.2", 
                                    "rec.dlp.particles.end_point.0", "rec.dlp.particles.end_point.1", "rec.dlp.particles.end_point.2",
                                    "rec.dlp_true.particles.start_dir.0", "rec.dlp_true.particles.start_dir.1", "rec.dlp_true.particles.start_dir.2",
                                    "rec.dlp.particles.start_dir.0", "rec.dlp.particles.start_dir.1", "rec.dlp.particles.start_dir.2",
                                    "rec.dlp_true.particles.ke", "rec.dlp.particles.ke",
                                    "rec.dlp.particles.mcs_ke", "rec.dlp.particles.csda_ke", "rec.dlp.particles.calo_ke",
                                    "rec.dlp.particles.length", "rec.dlp_true.particles.length", 
                                    ], library="np"))

    for col in dic:
        dic[col] = dic[col].tolist()
    df = pd.DataFrame(dic)

    # rename columns
    df.rename(columns={
        "rec.dlp_true.nu_id": "true_interaction_is_neutrino",
        "rec.dlp_true.is_fiducial": "true_interaction_is_fiducial",
        "rec.dlp_true.particles.interaction_id": "true_particle_interaction_id",
        "rec.dlp_true.particles.match_ids": "true_particle_match_reco_particle_id",
        "rec.dlp.particles.id": "reco_particle_id",
        "rec.dlp_true.particles.pdg_code": "true_particle_pdg_code",
        "rec.dlp.particles.pdg_code": "reco_particle_pdg_code",
        "rec.dlp_true.particles.is_contained": "true_particle_contained",
        "rec.dlp.particles.is_contained": "reco_particle_contained",
        "rec.dlp_true.particles.is_primary": "true_particle_primary",
        "rec.dlp.particles.is_primary": "reco_particle_primary",
        "rec.dlp_true.particles.start_point.0": "true_start_point_x",
        "rec.dlp_true.particles.start_point.1": "true_start_point_y",
        "rec.dlp_true.particles.start_point.2": "true_start_point_z",
        "rec.dlp.particles.start_point.0": "reco_start_point_x",
        "rec.dlp.particles.start_point.1": "reco_start_point_y",
        "rec.dlp.particles.start_point.2": "reco_start_point_z",
        "rec.dlp_true.particles.end_point.0": "true_end_point_x",
        "rec.dlp_true.particles.end_point.1": "true_end_point_y",
        "rec.dlp_true.particles.end_point.2": "true_end_point_z",
        "rec.dlp.particles.end_point.0": "reco_end_point_x",
        "rec.dlp.particles.end_point.1": "reco_end_point_y",
        "rec.dlp.particles.end_point.2": "reco_end_point_z",
        "rec.dlp_true.particles.start_dir.0": "true_start_dir_x",
        "rec.dlp_true.particles.start_dir.1": "true_start_dir_y",
        "rec.dlp_true.particles.start_dir.2": "true_start_dir_z",
        "rec.dlp.particles.start_dir.0": "reco_start_dir_x",
        "rec.dlp.particles.start_dir.1": "reco_start_dir_y",
        "rec.dlp.particles.start_dir.2": "reco_start_dir_z",
        "rec.dlp_true.particles.ke": "true_ke",
        "rec.dlp.particles.ke": "reco_ke",
        "rec.dlp.particles.mcs_ke": "reco_mcs_ke",
        "rec.dlp.particles.csda_ke": "reco_csda_ke",
        "rec.dlp.particles.calo_ke": "reco_calo_ke",
        "rec.dlp.particles.length": "reco_length",
        "rec.dlp_true.particles.length": "true_length",
    }, inplace=True)

    return df



In [4]:
def get_primary_fv_interactions_df(df):

    primary_fv_interactions = defaultdict(list)

    for event_i in tqdm(range(len(df))):
        row = df.iloc[event_i]
        nu_index = None
        for interaction_index in range(len(row["true_interaction_is_neutrino"])):
            interaction_type_id = row["true_interaction_is_neutrino"][interaction_index]
            if interaction_type_id >= 0:
                nu_index = interaction_index
                break
        if nu_index is None: # no true nu interaction, don't include this event
            continue
        new_true_particle_interaction_id = []
        new_true_particle_match_reco_particle_id = []
        new_true_particle_contained = []
        new_true_particle_primary = []
        new_true_particle_pdg_code = []
        new_true_particle_start_point_x = []
        new_true_particle_start_point_y = []
        new_true_particle_start_point_z = []
        new_true_particle_start_dir_x = []
        new_true_particle_start_dir_y = []
        new_true_particle_start_dir_z = []
        new_true_particle_end_point_x = []
        new_true_particle_end_point_y = []
        new_true_particle_end_point_z = []
        new_true_particle_ke = []
        new_true_particle_length = []

        new_matching_reco_particle_id = []
        new_matching_reco_particle_contained = []
        new_matching_reco_particle_primary = []
        new_matching_reco_particle_pdg_code = []
        new_matching_reco_particle_start_point_x = []
        new_matching_reco_particle_start_point_y = []
        new_matching_reco_particle_start_point_z = []
        new_matching_reco_particle_start_dir_x = []
        new_matching_reco_particle_start_dir_y = []
        new_matching_reco_particle_start_dir_z = []
        new_matching_reco_particle_end_point_x = []
        new_matching_reco_particle_end_point_y = []
        new_matching_reco_particle_end_point_z = []
        new_matching_reco_particle_ke = []
        new_matching_reco_particle_mcs_ke = []
        new_matching_reco_particle_csda_ke = []
        new_matching_reco_particle_calo_ke = []
        new_matching_reco_particle_length = []
        for true_particle_i in range(len(row["true_particle_interaction_id"])):

            if row["true_particle_interaction_id"][true_particle_i] != nu_index: # particle is not from a neutrino interaction
                continue

            if row["true_interaction_is_fiducial"][interaction_index] == 0: # interaction is not fiducial, don't include this particle
                continue

            new_true_particle_interaction_id.append(row["true_particle_interaction_id"][true_particle_i])
            new_true_particle_match_reco_particle_id.append(row["true_particle_match_reco_particle_id"][true_particle_i])
            new_true_particle_contained.append(row["true_particle_contained"][true_particle_i])
            new_true_particle_primary.append(row["true_particle_primary"][true_particle_i])
            new_true_particle_pdg_code.append(row["true_particle_pdg_code"][true_particle_i])
            new_true_particle_start_point_x.append(row["true_start_point_x"][true_particle_i])
            new_true_particle_start_point_y.append(row["true_start_point_y"][true_particle_i])
            new_true_particle_start_point_z.append(row["true_start_point_z"][true_particle_i])
            new_true_particle_start_dir_x.append(row["true_start_dir_x"][true_particle_i])
            new_true_particle_start_dir_y.append(row["true_start_dir_y"][true_particle_i])
            new_true_particle_start_dir_z.append(row["true_start_dir_z"][true_particle_i])
            new_true_particle_end_point_x.append(row["true_end_point_x"][true_particle_i])
            new_true_particle_end_point_y.append(row["true_end_point_y"][true_particle_i])
            new_true_particle_end_point_z.append(row["true_end_point_z"][true_particle_i])
            new_true_particle_ke.append(row["true_ke"][true_particle_i])
            new_true_particle_length.append(row["true_length"][true_particle_i])

            for reco_particle_i in range(len(row["reco_particle_id"])):
                if true_particle_i >= len(row["true_particle_match_reco_particle_id"]):
                    print("weird, too high true_particle_i index in event", event_i, "true_particle_i", true_particle_i)
                    continue
                if reco_particle_i >= len(row["reco_particle_id"]):
                    print("weird, too high reco_particle_i index in event", event_i, "reco_particle_i", reco_particle_i)
                    continue
                if row["reco_particle_id"][reco_particle_i] == row["true_particle_match_reco_particle_id"][true_particle_i]:
                    new_matching_reco_particle_id.append(row["reco_particle_id"][reco_particle_i])
                    new_matching_reco_particle_contained.append(row["reco_particle_contained"][reco_particle_i])
                    new_matching_reco_particle_primary.append(row["reco_particle_primary"][reco_particle_i])
                    new_matching_reco_particle_pdg_code.append(row["reco_particle_pdg_code"][reco_particle_i])
                    new_matching_reco_particle_start_point_x.append(row["reco_start_point_x"][reco_particle_i])
                    new_matching_reco_particle_start_point_y.append(row["reco_start_point_y"][reco_particle_i])
                    new_matching_reco_particle_start_point_z.append(row["reco_start_point_z"][reco_particle_i])
                    new_matching_reco_particle_start_dir_x.append(row["reco_start_dir_x"][reco_particle_i])
                    new_matching_reco_particle_start_dir_y.append(row["reco_start_dir_y"][reco_particle_i])
                    new_matching_reco_particle_start_dir_z.append(row["reco_start_dir_z"][reco_particle_i])
                    new_matching_reco_particle_end_point_x.append(row["reco_end_point_x"][reco_particle_i])
                    new_matching_reco_particle_end_point_y.append(row["reco_end_point_y"][reco_particle_i])
                    new_matching_reco_particle_end_point_z.append(row["reco_end_point_z"][reco_particle_i])
                    new_matching_reco_particle_ke.append(row["reco_ke"][reco_particle_i])
                    new_matching_reco_particle_mcs_ke.append(row["reco_mcs_ke"][reco_particle_i])
                    new_matching_reco_particle_csda_ke.append(row["reco_csda_ke"][reco_particle_i])
                    new_matching_reco_particle_calo_ke.append(row["reco_calo_ke"][reco_particle_i])
                    new_matching_reco_particle_length.append(row["reco_length"][reco_particle_i])
                    
        primary_fv_interactions["true_particle_interaction_id"].append(new_true_particle_interaction_id)
        primary_fv_interactions["true_particle_match_reco_particle_id"].append(new_true_particle_match_reco_particle_id)
        primary_fv_interactions["true_particle_contained"].append(new_true_particle_contained)
        primary_fv_interactions["true_particle_primary"].append(new_true_particle_primary)
        primary_fv_interactions["true_particle_pdg_code"].append(new_true_particle_pdg_code)
        primary_fv_interactions["true_start_point_x"].append(new_true_particle_start_point_x)
        primary_fv_interactions["true_start_point_y"].append(new_true_particle_start_point_y)
        primary_fv_interactions["true_start_point_z"].append(new_true_particle_start_point_z)
        primary_fv_interactions["true_start_dir_x"].append(new_true_particle_start_dir_x)
        primary_fv_interactions["true_start_dir_y"].append(new_true_particle_start_dir_y)
        primary_fv_interactions["true_start_dir_z"].append(new_true_particle_start_dir_z)
        primary_fv_interactions["true_end_point_x"].append(new_true_particle_end_point_x)
        primary_fv_interactions["true_end_point_y"].append(new_true_particle_end_point_y)
        primary_fv_interactions["true_end_point_z"].append(new_true_particle_end_point_z)
        primary_fv_interactions["true_ke"].append(new_true_particle_ke)
        primary_fv_interactions["true_length"].append(new_true_particle_length)
        primary_fv_interactions["matching_reco_particle_id"].append(new_matching_reco_particle_id)
        primary_fv_interactions["matching_reco_particle_contained"].append(new_matching_reco_particle_contained)
        primary_fv_interactions["matching_reco_particle_primary"].append(new_matching_reco_particle_primary)
        primary_fv_interactions["matching_reco_particle_pdg_code"].append(new_matching_reco_particle_pdg_code)
        primary_fv_interactions["matching_reco_particle_start_point_x"].append(new_matching_reco_particle_start_point_x)
        primary_fv_interactions["matching_reco_particle_start_point_y"].append(new_matching_reco_particle_start_point_y)
        primary_fv_interactions["matching_reco_particle_start_point_z"].append(new_matching_reco_particle_start_point_z)
        primary_fv_interactions["matching_reco_particle_start_dir_x"].append(new_matching_reco_particle_start_dir_x)
        primary_fv_interactions["matching_reco_particle_start_dir_y"].append(new_matching_reco_particle_start_dir_y)
        primary_fv_interactions["matching_reco_particle_start_dir_z"].append(new_matching_reco_particle_start_dir_z)
        primary_fv_interactions["matching_reco_particle_end_point_x"].append(new_matching_reco_particle_end_point_x)
        primary_fv_interactions["matching_reco_particle_end_point_y"].append(new_matching_reco_particle_end_point_y)
        primary_fv_interactions["matching_reco_particle_end_point_z"].append(new_matching_reco_particle_end_point_z)
        primary_fv_interactions["matching_reco_particle_ke"].append(new_matching_reco_particle_ke)
        primary_fv_interactions["matching_reco_particle_mcs_ke"].append(new_matching_reco_particle_mcs_ke)
        primary_fv_interactions["matching_reco_particle_csda_ke"].append(new_matching_reco_particle_csda_ke)
        primary_fv_interactions["matching_reco_particle_calo_ke"].append(new_matching_reco_particle_calo_ke)
        primary_fv_interactions["matching_reco_particle_length"].append(new_matching_reco_particle_length)
    primary_fv_interactions_df = pd.DataFrame({
        "true_particle_interaction_id": primary_fv_interactions["true_particle_interaction_id"],
        "true_particle_match_reco_particle_id": primary_fv_interactions["true_particle_match_reco_particle_id"],
        "true_particle_contained": primary_fv_interactions["true_particle_contained"],
        "true_particle_primary": primary_fv_interactions["true_particle_primary"],
        "true_particle_pdg_code": primary_fv_interactions["true_particle_pdg_code"],
        "true_start_point_x": primary_fv_interactions["true_start_point_x"],
        "true_start_point_y": primary_fv_interactions["true_start_point_y"],
        "true_start_point_z": primary_fv_interactions["true_start_point_z"],
        "true_start_dir_x": primary_fv_interactions["true_start_dir_x"],
        "true_start_dir_y": primary_fv_interactions["true_start_dir_y"],
        "true_start_dir_z": primary_fv_interactions["true_start_dir_z"],
        "true_end_point_x": primary_fv_interactions["true_end_point_x"],
        "true_end_point_y": primary_fv_interactions["true_end_point_y"],
        "true_end_point_z": primary_fv_interactions["true_end_point_z"],
        "true_ke": primary_fv_interactions["true_ke"],
        "true_length": primary_fv_interactions["true_length"],
        
        "matching_reco_particle_id": primary_fv_interactions["matching_reco_particle_id"],
        "matching_reco_particle_contained": primary_fv_interactions["matching_reco_particle_contained"],
        "matching_reco_particle_primary": primary_fv_interactions["matching_reco_particle_primary"],
        "matching_reco_particle_pdg_code": primary_fv_interactions["matching_reco_particle_pdg_code"],
        "matching_reco_particle_start_point_x": primary_fv_interactions["matching_reco_particle_start_point_x"],
        "matching_reco_particle_start_point_y": primary_fv_interactions["matching_reco_particle_start_point_y"],
        "matching_reco_particle_start_point_z": primary_fv_interactions["matching_reco_particle_start_point_z"],
        "matching_reco_particle_start_dir_x": primary_fv_interactions["matching_reco_particle_start_dir_x"],
        "matching_reco_particle_start_dir_y": primary_fv_interactions["matching_reco_particle_start_dir_y"],
        "matching_reco_particle_start_dir_z": primary_fv_interactions["matching_reco_particle_start_dir_z"],
        "matching_reco_particle_end_point_x": primary_fv_interactions["matching_reco_particle_end_point_x"],
        "matching_reco_particle_end_point_y": primary_fv_interactions["matching_reco_particle_end_point_y"],
        "matching_reco_particle_end_point_z": primary_fv_interactions["matching_reco_particle_end_point_z"],
        "matching_reco_particle_ke": primary_fv_interactions["matching_reco_particle_ke"],
        "matching_reco_particle_mcs_ke": primary_fv_interactions["matching_reco_particle_mcs_ke"],
        "matching_reco_particle_csda_ke": primary_fv_interactions["matching_reco_particle_csda_ke"],
        "matching_reco_particle_calo_ke": primary_fv_interactions["matching_reco_particle_calo_ke"],
        "matching_reco_particle_length": primary_fv_interactions["matching_reco_particle_length"],
    })

    return primary_fv_interactions_df


In [5]:
def get_true_muons_only_df(primary_fv_interactions_df):

    true_muons_only = defaultdict(list)

    for event_i in range(len(primary_fv_interactions_df)):
        row = primary_fv_interactions_df.iloc[event_i]
        nu_index = None
        num_found_muons = 0
        for true_particle_i in range(len(row["true_particle_pdg_code"])):
            if abs(row["true_particle_pdg_code"][true_particle_i]) == 13:
                num_found_muons += 1
                true_muons_only["true_particle_interaction_id"].append(row["true_particle_interaction_id"][true_particle_i])
                true_muons_only["true_particle_match_reco_particle_id"].append(row["true_particle_match_reco_particle_id"][true_particle_i])
                true_muons_only["true_particle_contained"].append(row["true_particle_contained"][true_particle_i])
                true_muons_only["true_particle_primary"].append(row["true_particle_primary"][true_particle_i])
                true_muons_only["true_particle_pdg_code"].append(row["true_particle_pdg_code"][true_particle_i])
                true_muons_only["true_start_point_x"].append(row["true_start_point_x"][true_particle_i])
                true_muons_only["true_start_point_y"].append(row["true_start_point_y"][true_particle_i])
                true_muons_only["true_start_point_z"].append(row["true_start_point_z"][true_particle_i])
                true_muons_only["true_start_dir_x"].append(row["true_start_dir_x"][true_particle_i])
                true_muons_only["true_start_dir_y"].append(row["true_start_dir_y"][true_particle_i])
                true_muons_only["true_start_dir_z"].append(row["true_start_dir_z"][true_particle_i])
                true_muons_only["true_end_point_x"].append(row["true_end_point_x"][true_particle_i])
                true_muons_only["true_end_point_y"].append(row["true_end_point_y"][true_particle_i])
                true_muons_only["true_end_point_z"].append(row["true_end_point_z"][true_particle_i])
                true_muons_only["true_ke"].append(row["true_ke"][true_particle_i])
                true_muons_only["true_length"].append(row["true_length"][true_particle_i])

                true_muons_only["matching_reco_particle_id"].append(row["matching_reco_particle_id"][true_particle_i])
                true_muons_only["matching_reco_particle_contained"].append(row["matching_reco_particle_contained"][true_particle_i])
                true_muons_only["matching_reco_particle_primary"].append(row["matching_reco_particle_primary"][true_particle_i])
                true_muons_only["matching_reco_particle_pdg_code"].append(row["matching_reco_particle_pdg_code"][true_particle_i])
                true_muons_only["matching_reco_particle_start_point_x"].append(row["matching_reco_particle_start_point_x"][true_particle_i])
                true_muons_only["matching_reco_particle_start_point_y"].append(row["matching_reco_particle_start_point_y"][true_particle_i])
                true_muons_only["matching_reco_particle_start_point_z"].append(row["matching_reco_particle_start_point_z"][true_particle_i])
                true_muons_only["matching_reco_particle_start_dir_x"].append(row["matching_reco_particle_start_dir_x"][true_particle_i])
                true_muons_only["matching_reco_particle_start_dir_y"].append(row["matching_reco_particle_start_dir_y"][true_particle_i])
                true_muons_only["matching_reco_particle_start_dir_z"].append(row["matching_reco_particle_start_dir_z"][true_particle_i])
                true_muons_only["matching_reco_particle_end_point_x"].append(row["matching_reco_particle_end_point_x"][true_particle_i])
                true_muons_only["matching_reco_particle_end_point_y"].append(row["matching_reco_particle_end_point_y"][true_particle_i])
                true_muons_only["matching_reco_particle_end_point_z"].append(row["matching_reco_particle_end_point_z"][true_particle_i])
                true_muons_only["matching_reco_particle_ke"].append(row["matching_reco_particle_ke"][true_particle_i])
                true_muons_only["matching_reco_particle_mcs_ke"].append(row["matching_reco_particle_mcs_ke"][true_particle_i])
                true_muons_only["matching_reco_particle_csda_ke"].append(row["matching_reco_particle_csda_ke"][true_particle_i])
                true_muons_only["matching_reco_particle_calo_ke"].append(row["matching_reco_particle_calo_ke"][true_particle_i])
                true_muons_only["matching_reco_particle_length"].append(row["matching_reco_particle_length"][true_particle_i])

    true_muons_only_df = pd.DataFrame({
        "true_particle_interaction_id": true_muons_only["true_particle_interaction_id"],
        "true_particle_match_reco_particle_id": true_muons_only["true_particle_match_reco_particle_id"],
        "true_particle_contained": true_muons_only["true_particle_contained"],
        "true_particle_primary": true_muons_only["true_particle_primary"],
        "true_particle_pdg_code": true_muons_only["true_particle_pdg_code"],
        "true_start_point_x": true_muons_only["true_start_point_x"],
        "true_start_point_y": true_muons_only["true_start_point_y"],
        "true_start_point_z": true_muons_only["true_start_point_z"],
        "true_start_dir_x": true_muons_only["true_start_dir_x"],
        "true_start_dir_y": true_muons_only["true_start_dir_y"],
        "true_start_dir_z": true_muons_only["true_start_dir_z"],
        "true_end_point_x": true_muons_only["true_end_point_x"],
        "true_end_point_y": true_muons_only["true_end_point_y"],
        "true_end_point_z": true_muons_only["true_end_point_z"],
        "true_ke": true_muons_only["true_ke"],
        "true_length": true_muons_only["true_length"],
        "matching_reco_particle_id": true_muons_only["matching_reco_particle_id"],
        "matching_reco_particle_contained": true_muons_only["matching_reco_particle_contained"],
        "matching_reco_particle_primary": true_muons_only["matching_reco_particle_primary"],
        "matching_reco_particle_pdg_code": true_muons_only["matching_reco_particle_pdg_code"],
        "matching_reco_particle_start_point_x": true_muons_only["matching_reco_particle_start_point_x"],
        "matching_reco_particle_start_point_y": true_muons_only["matching_reco_particle_start_point_y"],
        "matching_reco_particle_start_point_z": true_muons_only["matching_reco_particle_start_point_z"],
        "matching_reco_particle_start_dir_x": true_muons_only["matching_reco_particle_start_dir_x"],
        "matching_reco_particle_start_dir_y": true_muons_only["matching_reco_particle_start_dir_y"],
        "matching_reco_particle_start_dir_z": true_muons_only["matching_reco_particle_start_dir_z"],
        "matching_reco_particle_end_point_x": true_muons_only["matching_reco_particle_end_point_x"],
        "matching_reco_particle_end_point_y": true_muons_only["matching_reco_particle_end_point_y"],
        "matching_reco_particle_end_point_z": true_muons_only["matching_reco_particle_end_point_z"],
        "matching_reco_particle_ke": true_muons_only["matching_reco_particle_ke"],
        "matching_reco_particle_mcs_ke": true_muons_only["matching_reco_particle_mcs_ke"],
        "matching_reco_particle_csda_ke": true_muons_only["matching_reco_particle_csda_ke"],
        "matching_reco_particle_calo_ke": true_muons_only["matching_reco_particle_calo_ke"],
        "matching_reco_particle_length": true_muons_only["matching_reco_particle_length"],
    })

    return true_muons_only_df

In [6]:
# iterate over files in /nevis/riverside/data/leehagaman/spine/flat_cafs

combined_true_muons_only_df = pd.DataFrame()

for filename in os.listdir('/nevis/riverside/data/leehagaman/spine/flat_cafs'):
    if not filename.endswith('.flat.root'):
        print("skipping", filename)
        continue
    print("processing", filename)
    df = get_df_from_file(filename)
    primary_fv_interactions_df = get_primary_fv_interactions_df(df)
    true_muons_only_df = get_true_muons_only_df(primary_fv_interactions_df)

    combined_true_muons_only_df = pd.concat([combined_true_muons_only_df, true_muons_only_df])

print("done!")


processing input000.flat.root


  0%|          | 0/8832 [00:00<?, ?it/s]

processing input001.flat.root


  0%|          | 0/8735 [00:00<?, ?it/s]

processing input002.flat.root


...

processing input018.flat.root


  0%|          | 0/8838 [00:00<?, ?it/s]

processing input019.flat.root


  0%|          | 0/2362 [00:00<?, ?it/s]

done!


In [7]:
with open('true_muons_only_df.pkl', 'wb') as f:
    pickle.dump(combined_true_muons_only_df, f)
